In [1]:
import datetime
import shutil
import requests
import glob
import os

import zipfile
import glob
import warnings
import matplotlib.pyplot as plt
import pandas as pd

from multiprocessing import Pool, cpu_count
from functools import partial
from send_emails import send_txt_email

warnings.filterwarnings("ignore")

In [2]:
import sys
sys.stdout = open(f'IN_LOG_{datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")}.log','wt')

BASE_DIR = "/vol/bitbucket/pn222/satellite/mosdac"

In [3]:
cyclones_path = "./list_of_cyclones.xlsx"
df = pd.read_excel(cyclones_path)
df = df.drop('Unnamed: 8', axis=1)
insat_df = df[df["Satellite Data"] == "ISRO - INSAT"]
insat_df

,Region,Sub-Region,Satellite Data,Name,SSHWS Category - Peak,Form Date,Dissipated Date,Wikipedia Link
0,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Mocha,Category 5,09-05-2023,15-05-2023,https://en.wikipedia.org/wiki/Cyclone_Mocha
1,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Asani,Category 1,07-05-2022,12-05-2022,https://en.wikipedia.org/wiki/Cyclone_Asani
2,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Yaas,Category 1,23-05-2021,28-05-2021,https://en.wikipedia.org/wiki/Cyclone_Yaas
3,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Nivar,Category 1,23-10-2020,27-10-2020,https://en.wikipedia.org/wiki/Cyclone_Nivar
4,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Amphan,Category 5,16-05-2020,21-05-2020,https://en.wikipedia.org/wiki/Cyclone_Amphan
5,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Bulbul,Category 3,28-10-2019,31-10-2019,https://en.wikipedia.org/wiki/Cyclone_Matmo-Bu...
6,North Indian Ocean,Bay of Bengal,ISRO - INSAT,Fani,Category 5,26-04-2019,05-05-2019,https://en.wikipedia.org/wiki/Cyclone_Fani
8,North Indian Ocean,Arabian Sea & Bay of Bengal,ISRO - INSAT,Gulab - Shaheen,Category 1,24-09-2021,04-10-2021,https://en.wikipedia.org/wiki/Cyclones_Gulab_a...
9,North Indian Ocean,Arabian Sea,ISRO - INSAT,Tauktae,Category 4,14-05-2021,19-05-2021,https://en.wikipedia.org/wiki/Cyclone_Tauktae
10,North Indian Ocean,Arabian Sea,ISRO - INSAT,Nisarga,Category 2,01-06-2020,04-06-2020,https://en.wikipedia.org/wiki/Cyclone_Nisarga


In [4]:
import paramiko
import socket
import subprocess

def fetch_sftp_connection(hostname, port, username, password):
    sock = socket.socket(socket.AF_INET)
    sock.connect((hostname, port))
    transport = paramiko.Transport(sock)
    transport.connect(username=username, password=password)
    sftp = paramiko.SFTPClient.from_transport(transport)
    return sftp, transport

def close_sftp_connection(sftp, transport):
    sftp.close()
    transport.close()

def download_file_sftp(remote_path, local_path):
    sftp, transport = fetch_sftp_connection(hostname, port, username, password)
    sftp.get(remote_path, local_path)
    close_sftp_connection(sftp, transport)

def is_stub_already_present(dest_folder, stub):
    stubs = [x.split('/')[-1] for x in glob.glob(dest_folder+"*.h5")]
    if stub in stubs: 
        print(f"Present: {stub}")
        return True
    return False

def download_insat3d(date, order_no, name):
    year = date.year ; month = date.month ; day = date.day ; hour = date.hour ;
    
    stub = "3DIMG_" + f"{date.strftime('%d%b%Y_%H00').upper()}" + "_L1B_STD_V01R00.h5"
    remote_path = f"Order/{order_no}/{stub}"
    dest_folder = f"{BASE_DIR}/data/h5/{name.replace(' ', '').lower()}/{year}-{month:02}-{day:02}/" ; local_path  = f"{dest_folder}{stub}"
    os.makedirs(dest_folder, exist_ok=True)
    
    print(f'[{name}] - {date.strftime("%Y-%m-%d %H:%M")} - Downloading file ... ')
    try:
        if not is_stub_already_present(dest_folder, stub):
            download_file_sftp(remote_path, local_path)
        print(f'[{name}] - {date.strftime("%Y-%m-%d %H:%M")} - Downloaded.')
    except Exception as e:
        subprocess.run(f"rm -rf {local_path}", shell=True)
        print(f'[{name}] - {date.strftime("%Y-%m-%d %H:%M")} - Error: {e}')

In [5]:
ORDER_NO_MAPPINGS = {
    "Fani"  : "Jun23_086424",
    "Bulbul": "Jun23_086428"
}

In [12]:
%%time

hostname = 'download.mosdac.gov.in'
port = 22
username = 'ukp3jitnath'
password = 'Mosdac2@23'

for name in ORDER_NO_MAPPINGS.keys():    
    row = df.loc[df["Name"] == name].squeeze()
    order_no = ORDER_NO_MAPPINGS[name]
    start_date = datetime.datetime.strptime(row["Form Date"], "%d-%m-%Y")
    end_date = datetime.datetime.strptime(row["Dissipated Date"], "%d-%m-%Y") + datetime.timedelta(hours=23)
    
    current_date = start_date
    dates = [start_date]
    while current_date < end_date:
      current_date += datetime.timedelta(hours=1)
      dates.append(current_date)
    
    pool = Pool(cpu_count())
    download_func = partial(download_insat3d, order_no=order_no, name=name)
    results = pool.map(download_func, dates[-3:])
    pool.close()
    pool.join()
    
    print(f'[{name}] - All downloads are finished.')
          
    with open("IN_COMPLETE.txt", "a+") as file:
        file.write(f"{name}\t{datetime.datetime.now()}\n")
    
    subject = f"[COMPLETED] Download - Cyclone {name}"
    message_txt = f"""Download Completed"""
    send_txt_email(message_txt, subject)

In [ ]:
ORDER_NO_MAPPINGS = {
    "Fani"  : "Jun23_086424",
    "Bulbul": "Jun23_086428",
    "Mocha" : "Jun23_086446",
    "Asani" : "Jun23_086448",
    "Yaas"  : "Jun23_086450",
    "Nivar" : "Jun23_086452",
    "Amphan": "Jun23_086454",
    "Gulab - Shaheen"  : "Jun23_086472",
    "Tauktae" : "Jun23_086474",
    "Nisarga" : "Jun23_086476",
    "Maha"    : "Jun23_086478",
    "Kyarr"   : "Jun23_086480",
    "Vayu"    : "Jun23_086482"
}